In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import MultiStepLR

import os
import argparse

from models import *
from utils import progress_bar
from models.resnet import *

In [3]:
from torch.utils.tensorboard import SummaryWriter      
writer = SummaryWriter("./pyresnet18_batch128_3lr_log")

In [4]:
class args_():
    def __init__(self):
        self.lr=0.01
        self.n_epochs=120
        self.resume=False
args = args_()

In [5]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
        writer.add_scalar('Loss/train', (train_loss/(batch_idx+1)), epoch)
        writer.add_scalar('Accuracy/train', 100.*correct/total, epoch)
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            writer.add_scalar('Loss/test', (test_loss/(batch_idx+1)), epoch)
            writer.add_scalar('Accuracy/test', 100.*correct/total, epoch)
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


In [6]:



# parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
# parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
# parser.add_argument('--resume', '-r', action='store_true', help='resume from checkpoint')
# args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
# net = ResNet18()
net = torchvision.models.resnet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..


In [7]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)

# for epoch in range(start_epoch, start_epoch+args.n_epochs):
#     train(epoch)
#     test(epoch)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
milestones = [50, 100, 140]
scheduler = MultiStepLR(optimizer, milestones, gamma=0.1)

for epoch in range(start_epoch, start_epoch+args.n_epochs):
    scheduler.step()
    train(epoch)
    test(epoch)


Epoch: 0


/home/weihao_zhuang/anaconda3/envs/torch1/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:117: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


 [================================================================>]  Step: 264ms | Tot: 7s585ms | Loss: 1.723 | Acc: 39.676% (19838/5000 391/391 .....................................................]  Step: 19ms | Tot: 982ms | Loss: 2.514 | Acc: 24.159% (1608/665 52/391 ...................................................]  Step: 18ms | Tot: 1s1ms | Loss: 2.501 | Acc: 24.455% (1659/678 53/391 =>........................................................]  Step: 17ms | Tot: 1s38ms | Loss: 2.479 | Acc: 24.972% (1758/704 55/391 >.....................................................]  Step: 17ms | Tot: 1s354ms | Loss: 2.339 | Acc: 26.931% (2482/921 72/391 ===================================>................]  Step: 17ms | Tot: 5s488ms | Loss: 1.808 | Acc: 37.289% (13937/3737 292/391 ======================================>.......]  Step: 18ms | Tot: 6s503ms | Loss: 1.758 | Acc: 38.681% (17131/4428 346/391 =================================================>...]  Step: 18ms | Tot: 7s5ms | Loss: 1